In [11]:
import numpy as np
import open3d as o3d
import cv2
import os

current_dir = os.getcwd()

def load_camera_intrinsics(stereomap_file):
    cv_file = cv2.FileStorage(stereomap_file, cv2.FILE_STORAGE_READ)
    
    # Get camera matrix from stereoMap.xml
    camera_matrix = cv_file.getNode('cameraMatrixL').mat()
    
    # Extract intrinsic parameters
    fx = camera_matrix[0, 0]
    fy = camera_matrix[1, 1]
    cx = camera_matrix[0, 2]
    cy = camera_matrix[1, 2]
    
    cv_file.release()
    return fx, fy, cx, cy

def create_point_cloud(color_file, depth_file, stereomap_file):
    # Load camera intrinsics
    fx, fy, cx, cy = load_camera_intrinsics(stereomap_file)
    
    # Create Open3D intrinsics
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width=640, height=480,
        fx=fx, fy=fy,
        cx=cx, cy=cy
    )
    
    # Load color image
    color_raw = o3d.io.read_image(color_file)
    
    # Load and inspect depth data
    depth_raw = np.load(depth_file)
    print(f"Depth statistics:")
    print(f"  Shape: {depth_raw.shape}")
    print(f"  Valid points: {np.sum(depth_raw > 0)}")
    print(f"  Min depth: {np.min(depth_raw[depth_raw > 0]):.2f}")
    print(f"  Max depth: {np.max(depth_raw):.2f}")
    print(f"  Mean depth: {np.mean(depth_raw[depth_raw > 0]):.2f}")
    
    # Handle depth scaling more carefully
    valid_mask = (depth_raw > 0) & np.isfinite(depth_raw)
    
    if np.sum(valid_mask) == 0:
        print("ERROR: No valid depth points found!")
        return None
    
    # Determine appropriate depth scale
    max_depth = np.max(depth_raw[valid_mask])
    if max_depth > 100:  # Likely in mm
        depth_scale = 100.0
        depth_trunc = 220.0  # 8 meters
    else:  # Likely in meters  
        depth_scale = 1000.0
        depth_trunc = 8.0
    
    print(f"Using depth_scale: {depth_scale}, depth_trunc: {depth_trunc}")
    
    # Convert depth to uint16 format
    depth_scaled = depth_raw * depth_scale
    depth_scaled = np.clip(depth_scaled, 0, 65535)  # Clip to uint16 range
    depth_o3d = o3d.geometry.Image(depth_scaled.astype(np.uint16))
    
    # Create RGBD image with less aggressive filtering
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw, depth_o3d,
        depth_scale=depth_scale,
        depth_trunc=depth_trunc,  # Increased max depth
        convert_rgb_to_intensity=False
    )
    
    # Create point cloud
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        intrinsic
    )
    
    print(f"Point cloud created with {len(pcd.points)} points")
    
    # Flip the orientation to match standard coordinate system
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    
    return pcd

# def create_point_cloud(color_file, depth_file, stereomap_file):
#     # Load camera intrinsics
#     fx, fy, cx, cy = load_camera_intrinsics(stereomap_file)
    
#     # Create Open3D intrinsics
#     intrinsic = o3d.camera.PinholeCameraIntrinsic(
#         width=640, height=480,
#         fx=fx, fy=fy,
#         cx=cx, cy=cy
#     )
    
#     # Load color image
#     color_raw = o3d.io.read_image(color_file)
    
#     # Load depth data and convert to Open3D format
#     depth_raw = np.load(depth_file)  # Load the raw depth data
    
#     # Convert depth to uint16 format required by Open3D
#     depth_scale = 1000.0  # Scale factor to convert to millimeters
#     depth_o3d = o3d.geometry.Image(
#         (depth_raw * depth_scale).astype(np.uint16)
#     )
    
#     # Create RGBD image
#     rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
#         color_raw, depth_o3d,
#         depth_scale=depth_scale,
#         depth_trunc=5.0,  # Maximum depth in meters
#         convert_rgb_to_intensity=False
#     )
    
#     # Create point cloud
#     pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
#         rgbd_image,
#         intrinsic
#     )
    
#     # Flip the orientation to match standard coordinate system
#     pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    
#     return pcd

def main():
    current_dir = os.getcwd()
    stereomap_file = os.path.join(current_dir, "stereoMap.xml")
    
    # Specify your saved depth and color files
    color_file = os.path.join(current_dir, "images", "rect_L_color.png")
    depth_file = os.path.join(current_dir, "csre_stereo", "images", "depth_image.npy")

    # Create point cloud
    pcd = create_point_cloud(color_file, depth_file, stereomap_file)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])
    
    # Optionally save the point cloud
    o3d.io.write_point_cloud(f"pointcloud.ply", pcd)

if __name__ == "__main__":
    main()

Depth statistics:
  Shape: (480, 640)
  Valid points: 299664
  Min depth: 102.93
  Max depth: 4168.80
  Mean depth: 784.89
Using depth_scale: 100.0, depth_trunc: 220.0
Point cloud created with 48317 points
